In [1]:
# utilities
import re
import numpy as np
import pandas as pd
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from nltk.stem import WordNetLemmatizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
from collections import Counter
# Importing Dataset

DATASET_COLUMNS=['target','ids','date','flag','user','text']
DATASET_ENCODING = "ISO-8859-1"
df = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv', encoding=DATASET_ENCODING, names=DATASET_COLUMNS)
df.sample(5)

# Text Preprocessing

data=df[['text','target']]

data['target'] = data['target'].replace(4,1)

data['target'].unique()

data_pos = data[data['target'] == 1]
data_neg = data[data['target'] == 0]

data_pos = data_pos.iloc[:int(20000)]
data_neg = data_neg.iloc[:int(20000)]

dataset = pd.concat([data_pos, data_neg])

#dataset = data

# Install necessary libraries
# pip install transformers
# pip install torch
# pip install pandas


/tmp/ipykernel_19/4118638372.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['target'] = data['target'].replace(4,1)


In [2]:
import torch
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from tqdm import tqdm
import pandas as pd

# Assuming 'dataset' is your DataFrame
# Make sure the 'text' column exists in your dataset

# Load pre-trained XLNet model and tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased')

# Extract the 'text' column from the dataset
sentences = dataset['text'].tolist()

# Tokenize and process each sentence
tokenized_texts = [tokenizer.encode(sentence, add_special_tokens=True) for sentence in sentences]

# Pad sequences to have the same length
max_len = max(len(tokenized_text) for tokenized_text in tokenized_texts)
padded_tokenized_texts = [tokenized_text + [0] * (max_len - len(tokenized_text)) for tokenized_text in tokenized_texts]

# Convert to PyTorch tensors
input_ids = torch.tensor(padded_tokenized_texts)

# Set batch size
batch_size = 8

# Perform inference with the XLNet model in batches
predicted_sentiments = []

with torch.no_grad(), tqdm(total=len(input_ids)) as pbar:
    for i in range(0, len(input_ids), batch_size):
        batch_input_ids = input_ids[i:i + batch_size]
        batch_outputs = model(batch_input_ids)
        batch_sentiment_scores = batch_outputs.logits
        batch_sentiment_probabilities = torch.nn.functional.softmax(batch_sentiment_scores, dim=-1)
        batch_predicted_sentiments = torch.argmax(batch_sentiment_probabilities, dim=-1).tolist()
        predicted_sentiments.extend(batch_predicted_sentiments)
        pbar.update(len(batch_input_ids))

# Add predicted sentiments to the original dataset
dataset['predicted_sentiment_xlnet'] = predicted_sentiments

# Export the updated dataset to CSV
output_csv_path_xlnet = 'predicted_sentiments_dataset_xlnet.csv'
dataset.to_csv(output_csv_path_xlnet, index=False)

# Print a message indicating the export is complete
print(f'Dataset with predicted sentiments (XLNet) exported to {output_csv_path_xlnet}')

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 40000/40000 [2:45:18<00:00,  4.03it/s]


Dataset with predicted sentiments (XLNet) exported to predicted_sentiments_dataset_xlnet.csv
